# The Elite Ant System Algorithm For The Travelling Salesman Problem

Algorithm from the Book **Ant Colony Optimization by Marco Dorigo and Thomas Stützle**

In [1]:
#----------------------------------------
# Chia E. Tungom
# chemago99@yahoo.com
# July 30th 2020
# Shenzhen China
#----------------------------------------

In [2]:
# import libraries

import numpy as np
import math
import random

# Roulette Wheel for Probability

In [3]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    """ pi: list with probability values
        return roulette proportions"""
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    """ ppi: list of roulette proportions
        Pop: list of corresponding item   [item1, ... , itemN] """
    
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

# Calculate Route Cost

In [4]:
# calculate cost of each route
def Cost(Pop, graph):
    """ Pop: List of All Items in the e.g [[item1], ... , [itemN]]
    graph: cost matrix or graph 
    returns a cost vector for corresponding item """
    
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += graph[route[j]][route[j+1]]
        cost += graph[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Initialize Pheromone 

In [5]:
# AS initialization
def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

# EAS initialization
def EASTSPher(graph, e, m, rho):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    
    numer = e + m
    den = rho*cost
    pher = [[numer/den for i in range(n)] for j in range(n)]
    
    return pher

# Calculate Transition Probability

In [6]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        
    return connections

# Find Path 

In [7]:
def FindPath(phermatrix, graph, takeoff, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        prob = TransitionP(HP, alpha, beta)
        takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
        
        path.append(takeoff)
 
    return path

# Get Elite Route and Connections

In [8]:
def Elite(Elite, ant, graph):
    
    if Cost([ant], graph)[0] < Cost([Elite], graph)[0]:
        return ant
    else:
        return Elite

def connections(L):
    """L is a list
    one way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
    
    cons.append((L[-1], L[0]))
    
    return cons

def connectionsE(L):
    """L is a list
    Two way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
        cons.append( (L[i], L[i-1]) )
    
    cons.append((L[-1], L[0]))
    cons.append((L[0], L[1]))
    
    return cons

# Update Pheromone 

In [9]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix


def EASUpPheromone(route, EliteRoute, matrix, e):
    """route : route taken
    EliteRoute: Best Route found thus far
    matrix: pheromone matrix
    e: weighting factor for elite pheromone"""
    
    Tbs = connectionsE(EliteRoute)
    
    bstau = 1/Cost([EliteRoute], matrix)[0]
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        if (route[i], route[i+1]) in Tbs:
            matrix[route[i]][route[i+1]] = matrix[route[i]][route[i+1]] + tau + (e*bstau)
            matrix[route[i+1]][route[i]] = matrix[route[i+1]][route[i]] + tau + (e*bstau)
        else:
            matrix[route[i]][route[i+1]] += tau
            matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    if (route[-1], route[0]) in Tbs:
        matrix[route[-1]][route[0]] = matrix[route[-1]][route[0]] + tau + (e*bstau)
        matrix[route[0]][route[-1]] = matrix[route[0]][route[-1]] + tau + (e*bstau)
    else:
        matrix[route[-1]][route[0]] += tau
        matrix[route[0]][route[-1]] += tau

    return matrix

# Evaporate Pheromone 

In [10]:
def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = math.exp(-5)
    return matrix

def EvPheromoneNeg(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = (1-rho)*matrix[i][j]
        
    return matrix

# Generate a Toy Symmetric TSP Problem 

In [11]:
# A random graph
def Graph(dim, bounds):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([random.randint(bounds[0], bounds[1])]))  # route length
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Ant System Algorithm

In [12]:
def AS(Ngraph, ants, alpha = 1, beta = 3, rho = 0.5, e = 10, runs = 50, Pher = None):
    
    if Pher == None:
        #Pher = TSPher(Ngraph)
        Pher = EASTSPher(Ngraph, e, ants, rho)
        
    starthob = random.choice([i for i in range(len(Ngraph))])

    GOAT = [i for i in range(len(Ngraph))]  # initial best ant
    
    i = 0
    while i < runs:
        Ants = []
        #starthob = random.choice([i for i in range(len(Ngraph))])
        
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, alpha, beta)
            Ants.append(stof)
            GOAT = Elite(GOAT, stof, Ngraph)
   
        #Pheromone update
        for j in Ants:
            EASUpPheromone(j, GOAT, Pher, e)
            
        #Pheromone update  
        #EvPheromone(Pher, rho)
        EvPheromoneNeg(Pher, rho)
        
        if i%int(runs*0.05) == 0:
            C = Cost(Ants, Ngraph)
            E = Cost([GOAT], Ngraph)[0]
            print("Evolution ", i, " Elite Cost = ", E,  " Minimum Cost = ", min(C), "  and Maximum Cost = ", max(C))
            #print("<<<<<<<<  The Best Route IS  >>>>>>>> ", GOAT)
        i += 1 

    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), min(C) #, np.array(Pher)

# Example on the Toy Problem

In [13]:
# Toy Network graph 
D = 100
bounds = (2,10)
random.seed(10)
graph = Graph(D, bounds)

In [14]:
# Find Shortest Path with the Algorithm

ants = len(graph)
alpha = 1
beta = 3
rho = 0.5
e = len(graph)
runs = 200

AS(graph, ants, alpha, beta, rho, e, runs)

Evolution  0  Elite Cost =  270  Minimum Cost =  270   and Maximum Cost =  336
Evolution  10  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  230
Evolution  20  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  30  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  40  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  50  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  60  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  70  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  80  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  90  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  100  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  110  Elite Cost =  204  Minimum Cost =  204   and Maximum Cost =  204
Evolution  120  Elite Cost =  204  Mini

(array([[26, 95, 13, ..., 60, 43,  2],
        [26,  2, 43, ...,  1, 13, 95],
        [26, 95, 13, ..., 60, 43,  2],
        ...,
        [26, 95, 13, ..., 60, 43,  2],
        [26, 95, 13, ..., 60, 43,  2],
        [26, 95, 13, ..., 60, 43,  2]]), 204)

# Soving Real TSP problems 

In [15]:
# att48

import RouteMatrix as RM
path = './Data/att48.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 1
beta = 3
rho = 0.05
e = len(graph)
runs = 100

AS(graph, ants, alpha, beta, rho, e, runs)

Evolution  0  Elite Cost =  45090.276325932675  Minimum Cost =  45090.276325932675   and Maximum Cost =  67605.41688211176
Evolution  5  Elite Cost =  39277.72780514396  Minimum Cost =  39277.72780514396   and Maximum Cost =  57302.95063490428
Evolution  10  Elite Cost =  37018.80738755403  Minimum Cost =  37840.52058309438   and Maximum Cost =  48070.66704039239
Evolution  15  Elite Cost =  36688.15530266025  Minimum Cost =  36688.15530266025   and Maximum Cost =  50851.32751510507
Evolution  20  Elite Cost =  36288.75773804634  Minimum Cost =  36288.75773804634   and Maximum Cost =  46079.57083715013
Evolution  25  Elite Cost =  36288.75773804634  Minimum Cost =  36288.75773804634   and Maximum Cost =  43778.77947286677
Evolution  30  Elite Cost =  36288.75773804634  Minimum Cost =  36288.75773804634   and Maximum Cost =  44418.579542962005
Evolution  35  Elite Cost =  36288.75773804634  Minimum Cost =  36288.75773804634   and Maximum Cost =  47195.98102285946
Evolution  40  Elite Co

(array([[46, 12, 13, ..., 14, 11, 19],
        [46, 19, 11, ..., 24, 13, 12],
        [46, 12, 13, ..., 14, 11, 19],
        ...,
        [46, 19, 11, ..., 24, 13, 12],
        [46, 12, 13, ..., 14, 11, 19],
        [46, 12, 13, ..., 14, 11, 19]]), 35989.53344176988)

In [16]:
# burma14

import RouteMatrix as RM
path = './Data/burma14.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 1
beta = 3
rho = 0.05
e = len(graph)
runs = 1000

AS(graph, ants, alpha, beta, rho, e, runs)

Evolution  0  Elite Cost =  34.94959303391328  Minimum Cost =  34.94959303391328   and Maximum Cost =  54.34426262247057
Evolution  50  Elite Cost =  31.226915109427544  Minimum Cost =  31.456233837620537   and Maximum Cost =  43.24962068804266
Evolution  100  Elite Cost =  31.226915109427544  Minimum Cost =  31.456233837620537   and Maximum Cost =  38.51325695715954
Evolution  150  Elite Cost =  31.22691510942754  Minimum Cost =  31.226915109427544   and Maximum Cost =  32.531640412693264
Evolution  200  Elite Cost =  31.22691510942754  Minimum Cost =  31.226915109427544   and Maximum Cost =  31.456233837620537
Evolution  250  Elite Cost =  31.22691510942754  Minimum Cost =  31.226915109427544   and Maximum Cost =  31.456233837620537
Evolution  300  Elite Cost =  31.22691510942754  Minimum Cost =  31.226915109427544   and Maximum Cost =  32.531640412693264
Evolution  350  Elite Cost =  31.22691510942754  Minimum Cost =  31.22691510942754   and Maximum Cost =  31.226915109427544
Evolut

(array([[10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0],
        [10,  8,  9,  1, 13,  2,  3,  4,  5, 11,  6, 12,  7,  0]]),
 31.226915109427544)